# Building Block Assembly

> Building block assembly related functions

In [ ]:
#| default_exp building_block_assembly

In [ ]:
#| hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
from __future__ import annotations
from chem_templates.imports import *
from chem_templates.utils import *
from chem_templates.building_blocks import Synthon, BuildingBlock, ReactionGroup, ReactionUniverse
from chem_templates.template import Template, TemplateResult

/home/dmai/miniconda3/envs/chem_templates/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#| export

class AssemblyPool():
    def __init__(self, synthons: Optional[Synthon]=None):
        self.synthons = []
        self.mark_to_synthon = defaultdict(list)
        if synthons:
            for synthon in synthons:
                self.add_synthon(synthon)
        
    def add_synthon(self, synthon: Synthon):
        self.synthons.append(synthon)
        for mark in synthon.marks:
            self.mark_to_synthon[mark].append(synthon)
            
    def reaction_filter(self, rxn_universe: ReactionUniverse) -> AssemblyPool:
        valid = []
        for synthon in self.synthons:
            if rxn_universe.get_matching_reactions(synthon):
                valid.append(synthon)
                
        return AssemblyPool(valid)
    
    def get_matching(self, query_synthon: Synthon) -> list[Synthon]:
        matching_synthons = []
        for mark in query_synthon.compatible_marks:
            matching_synthons += self.mark_to_synthon[mark]
            
        return deduplicate_list(matching_synthons)

SyntaxError: invalid syntax (2126166158.py, line 4)

In [ ]:
#| export

class BuildingBlockNode():
    def __init__(self, name: str, template: Optional[Template]=None):
        self.name = name
        self.template = template
        
    def template_screen(self, synthon: Synthon, store_data: bool=True) -> TemplateResult:
        if self.template is not None:
            output = self.template(synthon)
        else:
            output = TemplateResult(True, [], [])
        
        if store_data:
            synthon.add_data({'template_data' : output, 'template_result' : output.result})
            
        return output

In [ ]:
#| export

class ReactionNode(BuildingBlockNode):
    def __init__(self, name: str, reaction_universe: ReactionUniverse):
        super().__init__(name, None)
        self.reaction_universe = reaction_universe

In [ ]:
#| export

class SynthonNode(BuildingBlockNode):
    def __init__(self, name: str, template: Optional[Template]=None):
        super().__init__(name, template)

In [ ]:
#| export

class ProductNode(BuildingBlockNode):
    def __init__(self, 
                 name: str, 
                 incoming_node: Union[SynthonNode, ProductNode], 
                 reaction_node: ReactionNode,
                 next_node: Union[SynthonNode, ProductNode],
                 template: Optional[Template]=None
                ):
        super().__init__(name, template)
        
        self.incoming_node = incoming_node
        self.reaction_node = reaction_node
        self.next_node = next_node

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

do we have bbs as inputs or synthons?
need to be able to trace back to initial bbs/synthons and reconstruct assembly inputs
assembly class to track construction?

synthon nodes - take in synthon. have template

constant nodes - constant synthon

reaction node - holds allowed reactions, incoming node (synthon or product), next synthon node

perhaps
assign pool of synthons to each node based on node template
synthon nodes can have a parent attr to grab reaction node for rxn screening?

synthon node:
    to setup:
        assign pool of possible BBs based on attachments / template
    during assembly:
        pushes pool upward
        
reaction node
    assembly:
        filter incoming pool by reactant to create (incoming, rxn) pairs
        
        
        
synthon pool
    list of current synthons
    dict of `mapping` to list of synthons
        ie grab synthon from pool A, get `compatible_marks`, look up `compatible_marks` on mapping dict in pool B,
        
        
assembly modes
    product (full combi)
    random
        randomly grab synthon from pool A
        randomly grab synthons from pool B until a pair is found that's compatible by the current rxn universe
        react
    both methods:
        build `chunksize` preassemblies in parallel
        go until exhausted or limit reached